In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [ ]:
nltk.download('punkt')

In [ ]:
ex = 'European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices'

In [ ]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [ ]:
sent = preprocess(ex)
sent

In [ ]:
pattern = 'NP: {<DT>?<JJ>*<NN>}'

In [ ]:
cp = nltk.RegexpParser(pattern)
cs = cp.parse(sent)
print(cs)

In [ ]:
from nltk.chunk import conlltags2tree, tree2conlltags
from pprint import pprint

iob_tagged = tree2conlltags(cs)
pprint(iob_tagged)

In [ ]:
ne_tree = ne_chunk(pos_tag(word_tokenize(ex)))
print(ne_tree)